## Build the environment

In [1]:
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly
!pip install --quiet --upgrade tensorflow-federated-nightly
!pip install --quiet --upgrade nest-asyncio
!pip install --quiet --upgrade tb-nightly  # or tensorboard, but not both

import nest_asyncio
nest_asyncio.apply()

Uninstalling tensorboard-2.4.1:
  Successfully uninstalled tensorboard-2.4.1
     |████████████████████████████████| 604kB 13.3MB/s 
     |████████████████████████████████| 448.5MB 40kB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 4.2MB 47.8MB/s 
     |████████████████████████████████| 194kB 57.3MB/s 
     |████████████████████████████████| 174kB 58.0MB/s 
     |████████████████████████████████| 471kB 44.2MB/s 
     |████████████████████████████████| 4.0MB 46.8MB/s 
     |████████████████████████████████| 1.3MB 43.4MB/s 
     |████████████████████████████████| 5.9MB 42.3MB/s 
     |████████████████████████████████| 4.9MB 47.5MB/s 
ERROR: tensorflow 2.4.1 requires tensorboard~=2.4, which is not installed.
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.37.1 which is incompatible.
ERROR: tensorflow 2.4.

In [2]:
%load_ext tensorboard

In [3]:
import collections
import os
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
AUTOTUNE = tf.data.experimental.AUTOTUNE
import math, numpy as np
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers


np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

## Prepare the data

### Input the data

we use tiny imagenet dataset from GitHub

Here's how we can load it.

In [4]:
# download the dataset from GitHub
! git clone https://github.com/Torch-Dragon/41004-AI-Capstone-Project-.git

Cloning into '41004-AI-Capstone-Project-'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 90 (delta 37), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (90/90), done.


In [5]:
! ls

41004-AI-Capstone-Project-  sample_data


In [6]:
#unzip dataset
! unzip -d ./Dataset ./41004-AI-Capstone-Project-/federated1.zip
! unzip -d ./Dataset ./41004-AI-Capstone-Project-/federated2.zip
! unzip -d ./Dataset ./41004-AI-Capstone-Project-/val+test.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: ./Dataset/client_16/5/5_352.jpg  
  inflating: ./Dataset/client_16/5/5_178.jpg  
  inflating: ./Dataset/client_16/5/5_595.jpg  
  inflating: ./Dataset/client_16/5/5_608.jpg  
  inflating: ./Dataset/client_16/5/5_350.jpg  
  inflating: ./Dataset/client_16/5/5_849.jpg  
  inflating: ./Dataset/client_16/5/5_321.jpg  
  inflating: ./Dataset/client_16/5/5_283.jpg  
  inflating: ./Dataset/client_16/5/5_913.jpg  
  inflating: ./Dataset/client_16/5/5_912.jpg  
  inflating: ./Dataset/client_16/5/5_245.jpg  
  inflating: ./Dataset/client_16/5/5_118.jpg  
  inflating: ./Dataset/client_16/5/5_73.jpg  
  inflating: ./Dataset/client_16/5/5_246.jpg  
  inflating: ./Dataset/client_16/5/5_81.jpg  
  inflating: ./Dataset/client_16/5/5_41.jpg  
  inflating: ./Dataset/client_16/5/5_270.jpg  
  inflating: ./Dataset/client_16/5/5_502.jpg  
  inflating: ./Dataset/client_16/5/5_702.jpg  
  inflating: ./Dataset/client_16/5/5_162.jpg  
  inflating: ./Dataset/client_16/5/5

### Explore the content of the data set.

In [7]:
import random
from pathlib import Path
import glob
# store the images paths to lists

train_paths = [glob.glob(r'/content/Dataset/client_'+str(i)+'/*/*.jpg') for i in range(20)]
valid_paths = glob.glob(r'/content/Dataset/val/*/*.jpg')
test_paths = glob.glob(r'/content/Dataset/test/*/*.jpg')


In [8]:
print(len(train_paths))

20


In [9]:
print(train_paths[0][0])

/content/Dataset/client_0/2/2_904.jpg


In [10]:
for i in range(20):
  random.shuffle(train_paths[i])
random.shuffle(valid_paths)
random.shuffle(test_paths)

In [11]:
train_paths[0][0].split('/')

['', 'content', 'Dataset', 'client_0', '9', '9_354.jpg']

In [12]:
valid_paths[0].split('/')

['', 'content', 'Dataset', 'val', '6', '6_64.jpg']

In [13]:
train_labels = [[int(path.split('/')[4]) for path in train_paths[i]] for i in range(20)]
valid_labels = [int(path.split('/')[4]) for path in valid_paths]
test_labels = [int(path.split('/')[4]) for path in test_paths]

In [14]:
print(train_labels[0][:10])
print(valid_labels[:10])
print(test_labels[:10])

[9, 6, 1, 6, 6, 4, 1, 1, 7, 4]
[6, 4, 1, 9, 8, 3, 0, 0, 1, 7]
[7, 9, 9, 7, 8, 8, 1, 0, 1, 2]


### Preprocessing the data set

Build a function to preprocess the dataset,  
`preprocess_image `can resize and normalize the input data set.

In [15]:
#The preprocess function about resizing and normalizing images and paths
def preprocess_train_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [64, 64]) #Resize all images
  #随机上下翻转
  image=tf.image.random_flip_left_right(image)
  #随机左右翻转
  image=tf.image.random_flip_up_down(image)
  #随机上下翻转图像
  image=tf.image.random_brightness(image,0.5)
  #随机改变对比度
  image=tf.image.random_contrast(image,0,1)
  
  #将图像进行归一化
   
  image /= 255.0  # normalize to [0,1] range

  return image

def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [64, 64]) #Resize all images
  image /= 255.0  # normalize to [0,1] range
  return image

def load_and_preprocess_train_image(path):
  image = tf.io.read_file(path)
  return preprocess_train_image(image)

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [16]:
# load and preprocess the images

train_images = [[load_and_preprocess_train_image(path) for path in train_paths[i]] 
                                           for i in range(20)]

valid_images = [load_and_preprocess_image(path) for path in valid_paths]

test_images = [load_and_preprocess_image(path) for path in test_paths]


# tranfer to numpy.array
for i in range(20): 
  train_images[i] = np.asarray(train_images[i])
  train_labels[i] = np.asarray(train_labels[i])

valid_images = np.asarray(valid_images)
valid_labels = np.asarray(valid_labels)

test_images = np.asarray(test_images)
test_labels = np.asarray(test_labels)



In [19]:
print("train_images[0][0]:")
print(train_images[0][0])
print("train_labels[0][0]:")
print(train_labels[0][0])
  

train_images[0][0]:
[[[0.6877806  0.61819637 0.5255928 ]
  [0.6865117  0.61692744 0.52432394]
  [0.6848198  0.6156586  0.523055  ]
  ...
  [0.77068365 0.7378981  0.69732046]
  [0.77406734 0.74001294 0.70070416]
  [0.77660525 0.74255085 0.70324206]]

 [[0.6865117  0.61692744 0.52432394]
  [0.68587726 0.61629295 0.5236895 ]
  [0.6850313  0.61587006 0.5232665 ]
  ...
  [0.7582059  0.7247859  0.68230486]
  [0.7563025  0.722248   0.68040144]
  [0.75503355 0.72097915 0.6791325 ]]

 [[0.6848198  0.6156586  0.523055  ]
  [0.6850313  0.61587006 0.5232665 ]
  [0.6852428  0.616434   0.52397144]
  ...
  [0.7403     0.7053997  0.66059226]
  [0.7328275  0.6979271  0.65311974]
  [0.7275403  0.69263995 0.6478325 ]]

 ...

 [[0.68608874 0.6101598  0.51882523]
  [0.6894725  0.6135437  0.52220905]
  [0.69264483 0.61777335 0.5262273 ]
  ...
  [0.6784047  0.6464652  0.58093196]
  [0.6748799  0.64653563 0.580791  ]
  [0.67255354 0.6473816  0.5805795 ]]

 [[0.6712846  0.5940868  0.50402117]
  [0.67509145 0.5

#Federated Learning

Question: We have used FedAvg here. But what do we need to do to improve the model performance if we implement Non-IID setting ?

In [17]:
BATCH_SIZE = 32
NUM_CLIENTS = 20
SAMPLE_CLIENTS = 10

def create_tf_dataset_for_client(client_id):
  client = tf.data.Dataset.from_tensor_slices((train_images[client_id], train_labels[client_id]))
  client = client.batch(BATCH_SIZE)
  client = client.prefetch(buffer_size=AUTOTUNE)
  return client

In [18]:
def federated_train_data():
  client_ids = np.random.choice(range(0,NUM_CLIENTS), size=SAMPLE_CLIENTS, replace=False)
  data= [create_tf_dataset_for_client(x)
  for x in client_ids
  ]
  return data

In [19]:
def federated_valid_data():
  valid_set = tf.data.Dataset.from_tensor_slices((valid_images, valid_labels))
  valid_set = valid_set.batch(BATCH_SIZE)
  valid_set = valid_set.prefetch(buffer_size=AUTOTUNE)
  data = [valid_set]
  return data

In [20]:
def federated_test_data():
  test_set = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
  test_set = test_set.batch(BATCH_SIZE)
  test_set = test_set.prefetch(buffer_size=AUTOTUNE)
  data = [test_set]
  return data

In [21]:
input_spec = federated_train_data()[0].element_spec

In [22]:
def create_keras_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(64, 64, 3)))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1000, activation='relu'))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  
  return model


Question: We have used SparseCategoricalCrossentropy which you did not suggest. And I have tried MSE and MAE. But they did not behave well. Would like to ask What loss function do you advise.

In [23]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [24]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=1.0))

In [25]:
str(iterative_process.initialize.type_signature)

'( -> <model=<trainable=<float32[3,3,3,32],float32[32],float32[3,3,32,64],float32[64],float32[12544,1000],float32[1000],float32[1000,10],float32[10]>,non_trainable=<>>,optimizer_state=<int64,float32[3,3,3,32],float32[32],float32[3,3,32,64],float32[64],float32[12544,1000],float32[1000],float32[1000,10],float32[10],float32[3,3,3,32],float32[32],float32[3,3,32,64],float32[64],float32[12544,1000],float32[1000],float32[1000,10],float32[10]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [26]:
#@test {"skip": true}
logdir = "/tmp/logs/scalars/training/"
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [27]:
evaluation = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
import time

NUM_ROUNDS = 50
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(NUM_ROUNDS):
    start = time.time()
    state, metrics = iterative_process.next(state, federated_train_data())
    end = time.time()
    valid_metrics = evaluation(state.model, federated_valid_data())

    train_metrics = {}
    train_metrics['accuracy'] = metrics['train']['sparse_categorical_accuracy']
    train_metrics['loss'] = metrics['train']['loss']
    train_metrics['val_accuracy'] = valid_metrics['sparse_categorical_accuracy']
    train_metrics['val_loss'] = valid_metrics['sparse_categorical_accuracy']
    
    for name, value in train_metrics.items():
      tf.summary.scalar(name, value, step=round_num+1)

    print('round', round_num+1, round(end-start), 's, ', 
          'accuracy=', metrics['train']['sparse_categorical_accuracy'],
          'loss=', metrics['train']['loss'],
          'val_accuracy=', valid_metrics['sparse_categorical_accuracy'],
          'val_loss=', valid_metrics['loss'])
                                                                              

In [ ]:
#@test {"skip": true}
!ls {logdir}
%tensorboard --logdir {logdir} --port=0

In [ ]:
#@test {"skip": true}
!rm -R /tmp/logs/scalars/*

In [ ]:
test_metrics = evaluation(state.model, federated_test_data())
print(test_metrics)

OrderedDict([('sparse_categorical_accuracy', 0.593), ('loss', 1.1702322)])
